In [78]:
import json
import time
import os
import collections

In [33]:
track_counts = collections.Counter()

data_path = '/Users/kwu/data/recsys2018/mpd.v1/data'

for fname in os.listdir(data_path):
    data = json.loads(open(os.path.join(data_path, fname)).read())
    
    for playlist in data['playlists']:
        for track in playlist['tracks']:
            track_id = track['track_uri'].split(':')[-1]
            track_counts[track_id] += 1  
            
with open('track_counts.json', 'w') as f:
    json.dump(dict(track_counts), f)

In [56]:
track_counts = collections.Counter(json.loads(open('track_counts.json').read()))

In [59]:
track_threshold = 100
common_track_counts =  collections.Counter({k:c for k, c in track_counts.items() if c > track_threshold})

In [68]:
valid_track_ids = set(common_track_counts.keys())
track_id_map = dict(zip(valid_track_ids, range(len(valid_track_ids))))
rev_track_id_map = dict(zip(track_id_map.values(), track_id_map.keys()))

In [79]:
playlist_threshold = 50
playlist_count = 0

playlist_id_map = {}

row_inds = []
col_inds = []

start = time.time()

for fname in os.listdir(data_path):
    data = json.loads(open(os.path.join(data_path, fname)).read())
    for playlist in data['playlists']:
        valid_tracks = [track_id_map[track['track_uri'].split(':')[-1]] for track in playlist['tracks'] if track['track_uri'].split(':')[-1] in valid_track_ids]
        if len(valid_tracks) > playlist_threshold:
            
            playlist_id_map[playlist_count] = playlist['pid']
            row_inds += [playlist_count] * len(valid_tracks)
            col_inds += valid_tracks
            
            playlist_count += 1
            
end = time.time()
print ("Time elapsed: {}s".format(round(end-start, 1)))

Time elapsed: 419.1s


In [81]:
from scipy import sparse

data = sparse.coo_matrix(([1] * len(row_inds), ))

39106490
39106490
